In [11]:
import pandas as pd

In [110]:
# base_path = "/blue/hcds_hackathon/hackathon_projects/lung_cancer_metabolomics/metabolite_data/"
base_path = ""
data1_path = base_path + "ST000368_1.txt"
data2_path = base_path + "ST000369.txt"

In [109]:
# function to convert file to pandas dataframe
def process_data(data_path, metabolite_offset):
    with open(data_path) as f:
        lines = f.read().split("\n")

    ## the first line is patient IDs
    keys = lines[0].split("\t")
    # print(len(keys))

    ## the second line is clinical data for each patient ID (organ/cancer/smoker/gender)
    clinical_data = [x.split(" | ") for x in lines[1].split("\t")]
    clinical_data = [([y.split(":")[len(y.split(":")) - 1] for y in x]) for x in clinical_data]
    # print(len(clinical_data))

    result = dict(zip(keys, clinical_data))
    
    ## the remaining lines are metabolomics
    ## error, some lines are NA. figure out where NA should be removed when attaching metabolomics
    for n in range(2, len(lines)):
        split_metabolics = lines[n].split("\t")
        zipped = list(zip(keys, split_metabolics))
        for x in zipped:
            result[x[0]].append(x[1])

    ## remove these rows as they are column names/identifiers for metabolites and not informative 
    metabolites = result.pop("Metabolite_name")
    refmet_name = result.pop("RefMet_name")
    keys.remove("Metabolite_name")
    keys.remove("RefMet_name")

    ## create column names and generate a dataframe to return
    colnames = ["Organ", "Cancer", "Smoker", "Gender"]
    for x in metabolites[metabolite_offset:]:
        colnames.append(x)
    df = pd.DataFrame(result).T
    df.columns = colnames

    return df

df1 = process_data(data1_path, 2)
df2 = process_data(data2_path, 2)
combined_df = pd.concat([df1, df2], ignore_index=True)

combined_df.to_csv("metabolites.csv")
combined_df.head()

,Organ,Cancer,Smoker,Gender,1-monoolein,1-monopalmitin,1-monostearin,2_3_5-trihydroxypyrazine NIST,2_3-dihydroxybutanoic acid NIST,2-deoxyerythritol,...,malonic acid,N-acetylmannosamine,octadecanol,p-cresol,phosphoethanolamine,ribitol,salicylaldehyde,sorbitol,stigmasterol,trihydroxypyrazine NIST
0,Plasma,Adenocarcinoma,Current,F,6799,165,107,67,34,74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Plasma,Adenocarcinoma,Current,F,17473,411,100,125,54,146,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Plasma,Adenocarcinoma,Current,F,38267,525,195,209,45,183,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Plasma,Adenocarcinoma,Current,F,12027,726,122,200,107,152,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Plasma,Adenocarcinoma,Current,F,19565,386,108,108,55,202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
